In [6]:
from osgeo import gdal, osr, ogr
import os

In [18]:
epsg_from=4326
epsg_to=3413

osng = osr.SpatialReference ()
osng.ImportFromEPSG ( epsg_to )
wgs84 = osr.SpatialReference ()
wgs84.ImportFromEPSG ( epsg_from )
tx = osr.CoordinateTransformation ( wgs84, osng )
    

In [24]:
tx.TransformPoint(70, -135)

(-2187927.649279021, -1.3397192962277733e-10, 0.0)

In [42]:
def reproject_dataset ( dataset, \
            pixel_spacing=10., epsg_from=4326, epsg_to=3413 ):
    """
    A sample function to reproject and resample a GDAL dataset from within 
    Python. The idea here is to reproject from one system to another, as well
    as to change the pixel size. The procedure is slightly long-winded, but
    goes like this:
    
    1. Set up the two Spatial Reference systems.
    2. Open the original dataset, and get the geotransform
    3. Calculate bounds of new geotransform by projecting the UL corners 
    4. Calculate the number of pixels with the new projection & spacing
    5. Create an in-memory raster dataset
    6. Perform the projection
    """
    # Define the UK OSNG, see <http://spatialreference.org/ref/epsg/27700/>
    osng = osr.SpatialReference ()
    osng.ImportFromEPSG ( epsg_to )
    wgs84 = osr.SpatialReference ()
    wgs84.ImportFromEPSG ( epsg_from )
    tx = osr.CoordinateTransformation ( wgs84, osng )
    
    # Up to here, all  the projection have been defined, as well as a 
    # transformation from the from to the  to :)
    # We now open the dataset
    g = gdal.Open ( dataset )
    # Get the Geotransform vector
    geo_t = g.GetGeoTransform ()
    print(geo_t)
    x_size = g.RasterXSize # Raster xsize
    y_size = g.RasterYSize # Raster ysize
    print(x_size)
    print(y_size)
    # Work out the boundaries of the new dataset in the target projection
    (ulx, uly, ulz ) = tx.TransformPoint(geo_t[3], geo_t[0])
    
    (lrx, lry, lrz ) = tx.TransformPoint( geo_t[3] + geo_t[5]*y_size, \
                                         geo_t[0] + geo_t[1]*x_size)
    
    print(ulx, uly, ulz, lrx, lry, lrz)
    # See how using 27700 and WGS84 introduces a z-value!
    # Now, we create an in-memory raster
    mem_drv = gdal.GetDriverByName( 'MEM' )
    # The size of the raster is given the new projection and pixel spacing
    # Using the values we calculated above. Also, setting it to store one band
    # and to use Float32 data type.
    dest = mem_drv.Create('dest', int((lrx - ulx)/pixel_spacing), \
            int((uly - lry)/pixel_spacing), 1, gdal.GDT_Float32)
    
    print(type(mem_drv))
    # Calculate the new geotransform
    new_geo = ( ulx, pixel_spacing, geo_t[2], \
                uly, geo_t[4], -pixel_spacing )
    # Set the geotransform
#     dest.SetProjection ( osng.ExportToWkt() )
#     dest.SetGeoTransform( new_geo )
    # Perform the projection/resampling 
    res = gdal.ReprojectImage( g, dest, \
                wgs84.ExportToWkt(), osng.ExportToWkt(), \
                gdal.GRA_Bilinear )
    return dest

In [41]:
help(gdal.ReprojectImage)

Help on function ReprojectImage in module osgeo.gdal:

ReprojectImage(*args, **kwargs)
    ReprojectImage(Dataset src_ds, Dataset dst_ds, char const * src_wkt=None, char const * dst_wkt=None, GDALResampleAlg eResampleAlg=GRA_NearestNeighbour, double WarpMemoryLimit=0.0, double maxerror=0.0, GDALProgressFunc callback=0, void * callback_data=None, char ** options=None) -> CPLErr



In [43]:
if __name__ == "__main__":
    
    source_root = r"C:\Users\mouju\Desktop\film\geomorph_data"
    target_root = r"C:\Users\mouju\Desktop\film\geomorph_data_epsg3413"
    if not os.path.exists(target_root):
        os.makedirs(target_root) 
        
    file_list = os.listdir(source_root)
    
    for file in file_list:
        source = os.path.join(source_root, file)
        print(source)
        # Now, reproject and resample the NDVI dataset
        reprojected_dataset = reproject_dataset (source)
        # This is a GDAL object. We can read it
        reprojected_data = reprojected_dataset.ReadAsArray ()
        # Let's save it as a GeoTIFF.
        driver = gdal.GetDriverByName ( "GTiff" )
        
        target = os.path.join(target_root, file)
        dst_ds = driver.CreateCopy(target, reprojected_dataset, 0 )
        dst_ds = None # Flush the dataset to disk
    

C:\Users\mouju\Desktop\film\geomorph_data\aspect-cosine_90M_n65w135.tif
(-135.0, 0.0008333333333333334, 0.0, 70.0, 0.0, -0.0008333333333333334)
6000
6000
-2187927.649279021 -1.3397192962277733e-10 0.0 -2739782.147388793 -239699.87842723154 0.0
<class 'osgeo.gdal.Driver'>


ValueError: Received a NULL pointer.